# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats

## 1.Завантаження залежних та незалежних вибірок з текстових файлів. 

In [134]:
path = "../data_lab4/dep/norm!=1.txt"
columns = ['x','y']
df = pd.read_csv(path, delimiter=" ", header=None, names=columns)
path = 
df.head()

,x,y
0,10.6972,5.1307
1,12.6110,4.6356
2,15.0273,6.8693
3,10.3691,5.9564
4,7.8548,5.8500


### check whether path contain depended

## Creating class for calculation quantiles

In [17]:
# Коефіцієнти для формули
C0 = 2.515_517
C1 = 0.802_853
C2 = 0.010_328
D1 = 1.432_788
D2 = 0.189_265_9
D3 = 0.001_308

class Quantile:
    
    @classmethod
    def calculate_quantile(cls, p):
        if p > 0.5:
            t = np.sqrt(-2 * np.log(1 - p))
            quantile = (t - ((C0 + C1 * t + C2 * t ** 2) / (1 + D1 * t + D2 * t ** 2 + D3 * t ** 3)))
        else:
            t = np.sqrt(-2 * np.log(p))
            quantile = -(t - ((C0 + C1 * t + C2 * t ** 2) / (1 + D1 * t + D2 * t ** 2 + D3 * t ** 3)))

        return quantile
        
    @staticmethod
    def student_quantile(p, v):
        up = Quantile.calculate_quantile(p)
        tpv = up + (1 / v) * (1 / 4) * (up ** 3 + up) + (1 / v ** 2) * (1 / 96) * (
            5 * up ** 5 + 16 * up ** 3 + 3 * up) + (1 / v ** 3) * (1 / 384) * (
                  3 * up ** 7 + 19 * up ** 5 + 17 * up ** 3 - 15 * up) + (1 / v ** 4) * (1 / 92_160) * (
                  79 * up ** 9 + 779 * up ** 7 + 1_482 * up ** 5 - 1_920 * up ** 3 - 945 * up)
        return tpv
    
    @staticmethod
    def fisher_quantile(p ,v1, v2):
        up = Quantile.calculate_quantile(p)   

        sigma = 1/v1 + 1/v2
        delta = 1/v1 - 1/v2
        
        z = (up * np.sqrt(sigma / 2) -
         (1 / 6) * delta * (up**2 + 2) +
         np.sqrt(sigma / 2) * (sigma / 24 * (up**2 + 3 * up) + (1 / 72) * (delta**2 / sigma) * (up**3 + 11 * up)) -
         (sigma * delta / 120) * (up**4 + 9 * up**2 + 8) +
         (delta**3 / (3240 * sigma)) * (3 * up**4 + 7 * up**2 - 16) +
         np.sqrt(sigma / 2) * (sigma**2 / 1920 * (up**5 + 20 * up**3 + 15 * up)) +
         (delta**4 / 2880) * (up**5 + 44 * up**3 + 183 * up) +
         (delta**5 / (155520 * sigma**2)) * (9 * up**5 + 284 * up**3 + 1513 * up)
        )

        return np.exp(2*z)

## Критерії перевірки однорідності двох залежних вибірок: критерії перевірки рівності дисперсій та середніх + критерій знакових рангів Вілкоксона

In [52]:
class DependentSamples:
    @classmethod
    def create_thrird_sample(cls, df):
        df['z'] = df['x']-df['y']

        z_mean = np.mean(df['z'])
        z_std = np.std(df['z'], ddof=1)
       
        return DependentSamples.calculate_t(len(df)-1, z_mean, z_std)

    @classmethod
    def calculate_t(cls, length, z_mean, z_std):
        if z_mean==0 and z_std==0:
            t=0
        else:
            t = z_mean*np.sqrt(length)/z_std
        return DependentSamples.is_mean_equal(t, length-1)

    @classmethod
    def is_mean_equal(cls, t, n):
        print('t equals: '+str(t))
        print('Quantile: '+str(Quantile.student_quantile(1-0.05/2, n)))
        return np.abs(t)<=Quantile.student_quantile(1-0.05/2, n)

    # @classmethod
    # def is_variances_equal(cls, t, n):
    #     print('t equals: '+str(t))
    #     print('Quantile: '+str(Quantile.student_quantile(1-0.05/2, n)))
    #     return np.abs(t)<=Quantile.student_quantile(1-0.05/2, n)
    
    @classmethod
    def calculate_statistic(cls, df):
        squared_x_std = np.std(df['x'], ddof=1)**2
        squared_y_std = np.std(df['y'], ddof=1)**2
        if(squared_x_std>=squared_y_std):
            f=squared_x_std/squared_y_std
            v1 = df['x'].count() - 1
            v2 = df['y'].count() - 1 
        else:
            f=squared_y_std/squared_x_std
            v1 = df['y'].count() - 1 
            v2 = df['x'].count() - 1
        
        #check is variances coincide
        f_fisher = Quantile.fisher_quantile(0.95,v1,v2)
        print('F:' +str(f))
        print('F fisher:' +str(f_fisher))
        return f<=f_fisher
        
    @staticmethod
    def check_for_homogeneity(df):
        mean_equal = DependentSamples.create_thrird_sample(df)
        varians_equal = DependentSamples.calculate_statistic(df)
        print('Is mean equals: '+str(mean_equal))
        print('Is varians equals: '+str(varians_equal))

    @staticmethod
    def check_for_homogeneity(df):
        mean_equal = DependentSamples.create_thrird_sample(df)
        varians_equal = DependentSamples.calculate_statistic(df)
        print('Is mean equals: '+str(mean_equal))
        print('Is varians equals: '+str(varians_equal))
        if mean_equal and varians_equal:
            print('homogeneous sample')
        else:
            print('NOT homogeneous sample')

#### Test

In [57]:
DependentSamples.check_for_homogeneity(df)

t equals: 25.714023078404278
Quantile: 1.9683892292999465
F:4.734445461528674
F fisher:1.2098984878687653
Is mean equals: False
Is varians equals: False
NOT homogeneous sample


## Критерії перевірки однорідності двох незалежних вибірок: критерії перевірки рівності дисперсій та середніх + ранговий критерій, заданий індивідуальним варіантом.

In [117]:
class IndependentSamples:
    @classmethod
    def calculate_weighted_average_and_t(cls, df):
        x_mean = np.mean(df['x'])
        y_mean = np.mean(df['y'])
        x_dispersion = np.std(df['x'], ddof=1)**2
        y_dispersion = np.std(df['y'], ddof=1)**2
        x_n =df['x'].count()
        y_n =df['y'].count()

        weighted_average = ((x_n-1)*x_dispersion+(y_n-1)*y_dispersion)/(x_n+y_n-2)
        
        t = (x_mean-y_mean)/(np.sqrt(weighted_average/x_n+weighted_average/y_n))
        
        print('weighted average: '+str(weighted_average))
        print('t equals: '+str(t))
        
        return IndependentSamples.is_mean_equal(t, x_n+y_n-2)

    @classmethod
    def is_mean_equal(cls, t, v):
        print('Quantile: '+str(Quantile.student_quantile(1-0.05/2, v)))
        return np.abs(t)<=Quantile.student_quantile(1-0.05/2, v)

    @classmethod
    def calculate_criterion_with_welch_correction(cls, df):
        x_mean = np.mean(df['x'])
        y_mean = np.mean(df['y'])
        x_dispersion = np.std(df['x'], ddof=1)**2
        y_dispersion = np.std(df['y'], ddof=1)**2
        x_n =df['x'].count()
        y_n =df['y'].count()
        
        t = (x_mean-y_mean)/(np.sqrt(x_dispersion/x_n+y_dispersion/y_n))
        
        v = (x_dispersion/x_n+y_dispersion/y_n)**2*((1/(x_n-1)*(x_dispersion/x_n)**2)+(1/(y_n-1)*(y_dispersion/y_n)**2))**-1

        print('v: '+str(v))
        print('t equals: '+str(t))
        
        return IndependentSamples.is_mean_equal(t, v)

    @classmethod
    def claculate_rank_shift_criterion(cls, df, n1, n2):
        v = sum(list(df))
        ev = 1/2*n1*n2
        dv = 1/12*n1*n2*(n1+n2+1)
        u = (v-ev)/np.sqrt(dv)
        print('v equals: '+str(v))
        print('E{V} equals: '+str(ev))
        print('D{V} equals: '+str(dv))
        print('u equals: '+str(u))
        quantile = Quantile.calculate_quantile(1-0.05/2)
        print('Quantile: '+str(quantile))
        return np.abs(u)<=quantile
    
    @staticmethod
    def check_for_homogeneity(df, varians_equal):

        if varians_equal:
            mean_equal = IndependentSamples.calculate_weighted_average_and_t(df)
        else:
            mean_equal = IndependentSamples.calculate_criterion_with_welch_correction(df)

        print('Is mean equals: '+str(mean_equal))
        print('Is varians equals: '+str(varians_equal))
        if mean_equal and varians_equal:
            print('homogeneous sample')
        else:
            print('NOT homogeneous sample')
       

    @staticmethod
    def calculate_rank_with_mann_whitney(df):
        result = {}
        for x_val in df['x']:
            inversions = 0
            for y_val in df['y']:
                if y_val < x_val:
                    inversions += 1
                elif y_val == x_val:
                    inversions += 0.5  # semi inversion
            result[x_val] = inversions
        shift_exist = IndependentSamples.claculate_rank_shift_criterion(result.values(), df['x'].count(), df['y'].count())
        if shift_exist:
            print('NO shift exist')
        else:
            print('shift exist')
            
        return shift_exist

#### Test

In [135]:
print(''+str(IndependentSamples.check_for_homogeneity(df, DependentSamples.calculate_statistic(df))))
res = IndependentSamples.calculate_rank_with_mann_whitney(df)

F:4.734445461528674
F fisher:1.2098984878687653
v: 419.9140102725909
t equals: 26.043869255210645
Quantile: 1.9660608500811243
Is mean equals: False
Is varians equals: False
NOT homogeneous sample
None
v equals: 83616.5
E{V} equals: 45000.0
D{V} equals: 4507500.0
u equals: 18.188841619423197
Quantile: 1.960392179315197
shift exist


### Check wheather distribution normal or not

In [41]:
alpha = 0.05
# Shapiro-Wilk test for 'x'
statistic_x, p_value_x = stats.shapiro(df['x'])
normal_x = p_value_x > alpha

# Shapiro-Wilk test for 'y'
statistic_y, p_value_y = stats.shapiro(df['y'])
normal_y = p_value_y > alpha

if normal_x and normal_y:
    normal =True;
else:
    normal =False;

print(normal)

True


## Підрахунок і виведення у таблицю незсунених кількісних характеристики кожної виибірки, а також вибірки різниць у випадку залежних вибірок. 

### creating class for calculating a Quantitative Characteristics

In [136]:
class QuantitativeCharacteristics:

    def __init__(self,sample):
        self.sample = sample
        self.characteristic_fields_map = ["Mean", "Median", "Std", "Skewness", "Kurtosis", "Min", "Max"]
        self.characteristic_table_dictionary = dict()
        self.mean = None
        self.std_deviation = None
        self.a = None
        self.e = None
        self.sa = None
        self.se = None
    
    def add_to_dictionary(self, column_name, column_data):
        self.characteristic_table_dictionary.update({column_name: column_data})

    def display(self):
        self.add_characteristics()
        return self.characteristic_table_dictionary

    def add_characteristics(self):
        estimation = self.calculate_estimation()
        self.add_to_dictionary("Characteristic", self.characteristic_fields_map)
        self.add_to_dictionary("Estimation", estimation.keys())
        self.add_to_dictionary("SEM", estimation.values())
        confidence_intervals = self.calculate_confidence_interval(estimation)
        self.add_to_dictionary("95% Confidence Interval", confidence_intervals)

    def calculate_confidence_interval(self, estimation: dict):
        confidence_interval = list(range(0, 7))
        median_conf_interval = self.calculate_median_confidence_interval(list(estimation.keys())[1])
        confidence_interval[1] = f"[{median_conf_interval[0]}; {median_conf_interval[1]}]"
        confidence_interval[5] = "───"
        confidence_interval[6] = "───"

        v = len(self.sample) - 1
        student_quantile = Quantile.student_quantile(0.95, v)
        print("student_quantile: " + str(student_quantile))
        i = 0
        for key in estimation.keys():
            if (estimation.get(key) == "───"):
                i = i + 1
                continue
            upper_bound = round(key - student_quantile * estimation.get(key), 4)
            lower_bound = round(key + student_quantile * estimation.get(key), 4)
            confidence_interval[i] = f"[{upper_bound}; {lower_bound}]"

            i = i + 1

        return confidence_interval

    def calculate_median_confidence_interval(self, median):
        num_samples = 1000

        bootstrap_samples = [np.random.choice(self.sample, size=len(self.sample), replace=True) for _ in
                             range(num_samples)]

        bootstrap_medians = np.median(bootstrap_samples, axis=1)

        std_error_median = np.std(bootstrap_medians, ddof=1)
        v = len(self.sample) - 1
        student_quantile = Quantile.student_quantile(1 - 0.05 / 2, v)
        lower_bound = round(median - student_quantile * std_error_median, 4)
        upper_bound = round(median + student_quantile * std_error_median, 4)

        return lower_bound, upper_bound

    def calculate_normal_interval(self):
        v = len(self.sample) - 1
        student_quantile = Quantile.student_quantile(1 - 0.05 / 2, v)
        lower_bound = round(self.mean - student_quantile * self.std_deviation, 4)
        upper_bound = round(self.mean + student_quantile * self.std_deviation, 4)
        return lower_bound, upper_bound

    def calculate_estimation(self):
        estimation_map = dict()
        self.mean = round(np.mean(self.sample), 4)
        median = round(np.median(self.sample), 4)
        self.std_deviation = round(np.std(self.sample, ddof=1), 4)
        self.a = round(stats.skew(self.sample), 4)
        self.e = round(stats.kurtosis(self.sample), 4)
        minimum = round(min(self.sample), 4)
        maximum = round(max(self.sample), 4)

        n_len = len(self.sample)
        mean_std = round(self.std_deviation / np.sqrt(n_len), 4)
        std_std_deviation = round(self.std_deviation / np.sqrt(2 * n_len), 4)
        self.sa = round(np.sqrt(6 * n_len * (n_len - 1) / ((n_len - 2) * (n_len + 1) * (n_len + 3))), 4)
        self.se = round(
            np.sqrt(24 * n_len * (n_len - 1) ** 2 / ((n_len - 3) * (n_len - 2) * (n_len + 3) * (n_len + 5))), 4)

        estimation_map[self.mean] = mean_std
        estimation_map[median] = "───"
        estimation_map[self.std_deviation] = std_std_deviation
        estimation_map[self.a] = self.sa
        estimation_map[self.e] = self.se
        estimation_map[minimum] = "───"
        estimation_map[maximum] = "───"
        return estimation_map

#### Test

In [137]:
q_c1 = QuantitativeCharacteristics(df['x'])
q_c2 = QuantitativeCharacteristics(df['y'])
res1= pd.DataFrame(q_c1.display())
print(res1)
print('----------')
res2=  pd.DataFrame(q_c2.display())
print(res2)


student_quantile: 1.650323668219721
  Characteristic  Estimation     SEM 95% Confidence Interval
0           Mean     10.0984  0.1736       [9.8119; 10.3849]
1         Median     10.1878     ───        [9.8176; 10.558]
2            Std      3.0075  0.1228        [2.8048; 3.2102]
3       Skewness      0.0207  0.1407       [-0.2115; 0.2529]
4       Kurtosis     -0.1347  0.2805       [-0.5976; 0.3282]
5            Min      2.6526     ───                     ───
6            Max     20.3890     ───                     ───
----------
student_quantile: 1.650323668219721
  Characteristic  Estimation     SEM 95% Confidence Interval
0           Mean      5.1214  0.0798        [4.9897; 5.2531]
1         Median      5.0903     ───        [4.8848; 5.2958]
2            Std      1.3822  0.0564        [1.2891; 1.4753]
3       Skewness      0.2429  0.1407        [0.0107; 0.4751]
4       Kurtosis      0.5894  0.2805        [0.1265; 1.0523]
5            Min      1.6105     ───                     ───
6 

### Checking the equality of variances

In [10]:
x_mean = np.mean(df['x'])
y_mean = np.mean(df['y'])
squared_x_std = np.std(df['x'], ddof=1)**2
squared_y_std = np.std(df['y'], ddof=1)**2
print("Mean for 1 column: "+ str(x_mean))
print("Mean for 2 column: "+ str(y_mean))
print("STD for 1 column: "+ str(squared_x_std))
print("STD for 2 column: "+ str(squared_y_std))

Mean for 1 column: 19.944359420289857
Mean for 2 column: 0.5
STD for 1 column: 62.469627554837615
STD for 2 column: 0.2518248175182482


#### Calculate P value with CDF(using student quantile)

### Calculate paired t-test

In [11]:
v1 = df['x'].count() - 1
v2 = df['y'].count() - 1 

Quantile.fisher_quantile(0.95,v1,v2)


1.325667428104424

In [ ]:
F = (squared_x_std/squared_y_std)
if(normal):
    2
elif(f>1):
    print("F equalse: " + str(F))
    2